In [1]:
import pandas as pd
import os

--- Execução do Notebook Gold ---

In [2]:
# --- Configuração de Caminhos e Diretórios ---
path_silver = '../data/silver/dados_limpos.csv'
path_gold = '../data/gold/'
os.makedirs(path_gold, exist_ok=True)

In [3]:
# --- Carregar Dados da Camada Silver ---
try:
    df = pd.read_csv(path_silver)
    print(f"Dados limpos carregados: {df.shape}")
except FileNotFoundError:
    print(f"Erro: Arquivo '{path_silver}' não encontrado.")
    df = pd.DataFrame() # Cria um dataframe vazio para evitar que o notebook quebre

# Garante que o df não está vazio para prosseguir
if df.empty:
    print("DataFrame está vazio. Saindo do notebook.")
    exit()

Dados limpos carregados: (55682, 31)


In [4]:
# ==========================================
# AGREGAÇÃO 1: Produção Anual por Cultura
# ==========================================
print("Calculando: Produção Anual por Cultura...")
prod_anual = df.groupby(['year', 'crop']).agg(
    area_total_plantada=('area', 'sum'),
    producao_total=('production', 'sum')
).reset_index()

Calculando: Produção Anual por Cultura...


In [5]:
prod_anual['rendimento_ponderado'] = prod_anual.apply(
    lambda row: row['producao_total'] / row['area_total_plantada'] if row['area_total_plantada'] > 0 else 0,
    axis=1
)
prod_anual.to_csv(os.path.join(path_gold, 'producao_anual_cultura.csv'), index=False)

In [6]:
# ==========================================
# AGREGAÇÃO 2: Desempenho por Região e Cultura
# ==========================================
print("Calculando: Desempenho por Região e Cultura...")
desempenho_regiao = df.groupby(['state_district', 'crop']).agg(
    area_total=('area', 'sum'),
    producao_total=('production', 'sum'),
    rendimento_medio_local=('yield', 'mean')
).reset_index()
desempenho_regiao = desempenho_regiao.sort_values(by='producao_total', ascending=False)
desempenho_regiao.to_csv(os.path.join(path_gold, 'desempenho_regiao_cultura.csv'), index=False)

Calculando: Desempenho por Região e Cultura...


In [7]:
# ==========================================
# AGREGAÇÃO 3: Análise Sazonal e Climática
# ==========================================
print("Calculando: Análise Sazonal e Climática...")
analise_sazonal = df.groupby(['season', 'crop']).agg(
    producao_media=('production', 'mean'),
    rendimento_medio=('yield', 'mean'),
    area_media=('area', 'mean'),
    temp_media_jun_sep=('temp_jun_sep', 'mean'),
    precipitacao_media_jun_sep=('jun_sep', 'mean')
).reset_index()
analise_sazonal = analise_sazonal.sort_values(by='rendimento_medio', ascending=False)
analise_sazonal.to_csv(os.path.join(path_gold, 'analise_sazonal_clima.csv'), index=False)

Calculando: Análise Sazonal e Climática...


In [8]:
# ==========================================
# AGREGAÇÃO 4: Tendência de Rendimento Ano-a-Ano (Growth)
# ==========================================
print("Calculando: Tendência de Rendimento (Yield) Ano-a-Ano por Cultura...")
trend_rendimento = df.groupby(['crop', 'year']).agg(
    rendimento_medio=('yield', 'mean')
).reset_index()
trend_rendimento = trend_rendimento.sort_values(by=['crop', 'year'])
trend_rendimento['crescimento_anual_rendimento_percent'] = trend_rendimento.groupby('crop')['rendimento_medio'].pct_change() * 100
trend_rendimento['crescimento_anual_rendimento_percent'] = trend_rendimento['crescimento_anual_rendimento_percent'].fillna(0)
trend_rendimento.to_csv(os.path.join(path_gold, 'tendencia_anual_rendimento.csv'), index=False)

Calculando: Tendência de Rendimento (Yield) Ano-a-Ano por Cultura...


In [9]:
# ==========================================
# AGREGAÇÃO 5: Benchmark de Desempenho Regional (vs. Média Nacional)
# ==========================================
print("Calculando: Benchmark de Desempenho Regional por Cultura...")
media_nacional_cultura = df.groupby('crop')['yield'].mean().to_frame(name='rendimento_medio_nacional').reset_index()
media_regional_cultura = df.groupby(['state_district', 'crop']).agg(
    rendimento_medio_local=('yield', 'mean'),
    producao_total_local=('production', 'sum')
).reset_index()
benchmark_regional = pd.merge(
    media_regional_cultura,
    media_nacional_cultura,
    on='crop'
)
benchmark_regional['diferenca_vs_media_percent'] = (
    (benchmark_regional['rendimento_medio_local'] - benchmark_regional['rendimento_medio_nacional']) /
     benchmark_regional['rendimento_medio_nacional']
) * 100
benchmark_regional = benchmark_regional.sort_values(by='diferenca_vs_media_percent', ascending=False)
benchmark_regional.to_csv(os.path.join(path_gold, 'benchmark_regional_rendimento.csv'), index=False)

Calculando: Benchmark de Desempenho Regional por Cultura...


In [10]:
# ==========================================
# AGREGAÇÃO 6: Perfil Climático por Região e Cultura
# ==========================================
print("Calculando: Perfil Climático Médio por Região e Cultura...")
perfil_climatico = df.groupby(['state_district', 'crop']).agg(
    rendimento_medio=('yield', 'mean'),
    precipitacao_anual_media=('ann', 'mean'),
    precipitacao_jun_sep_media=('jun_sep', 'mean'),
    temp_media_janfeb=('temp_jan_feb', 'mean'),
    temp_media_marmay=('temp_mar_may', 'mean'),
    temp_media_junsep=('temp_jun_sep', 'mean'),
    temp_media_octdec=('temp_oct_dec', 'mean')
).reset_index()
perfil_climatico.to_csv(os.path.join(path_gold, 'perfil_climatico_regiao_cultura.csv'), index=False)

Calculando: Perfil Climático Médio por Região e Cultura...


In [11]:
# ==========================================
# AGREGAÇÃO 7: Análise de Volatilidade (Risco) do Rendimento
# ==========================================
print("Calculando: Análise de Volatilidade (Risco) do Rendimento...")
volatilidade_df = df.groupby(['state_district', 'crop']).agg(
    rendimento_medio=('yield', 'mean'),
    rendimento_std_dev=('yield', 'std'),
    contagem_registros=('yield', 'count')
).reset_index()

Calculando: Análise de Volatilidade (Risco) do Rendimento...


In [12]:
min_registros = 3
volatilidade_df = volatilidade_df[volatilidade_df['contagem_registros'] >= min_registros].copy()
print(f"Filtrando volatilidade para grupos com {min_registros} ou mais registros.")

volatilidade_df['coef_variacao_rendimento_percent'] = (
    (volatilidade_df['rendimento_std_dev'] / volatilidade_df['rendimento_medio']) * 100
)
volatilidade_df = volatilidade_df.fillna(0)
volatilidade_df = volatilidade_df.sort_values(by='coef_variacao_rendimento_percent', ascending=False)
volatilidade_df.to_csv(os.path.join(path_gold, 'volatilidade_rendimento_regiao.csv'), index=False)

Filtrando volatilidade para grupos com 3 ou mais registros.
